# EGA

In [5]:
DIM = 100  # Number of bits in the bit strings (i.e. the "models").
NOISE_STDEV = 0.01  # Standard deviation of the simulated training noise.

class Model(object):
    
  def __init__(self):
    self.arch = None
    self.accuracy = None
    
  def __str__(self):
    """Prints a readable version of this bitstring."""
    return '{0:b}'.format(self.arch)
  
def train_and_eval(arch):
  """Simulates training and evaluation.
  
  Computes the simulated validation accuracy of the given architecture. See
  the `accuracy` attribute in `Model` class for details.
  
  Args:
    arch: the architecture as an int representing a bit-string.
  """
  accuracy =  float(_sum_bits(arch)) / float(DIM)
  accuracy += random.gauss(mu=0.0, sigma=NOISE_STDEV)
  accuracy = 0.0 if accuracy < 0.0 else accuracy
  accuracy = 1.0 if accuracy > 1.0 else accuracy
  return accuracy

def _sum_bits(arch):
  """Returns the number of 1s in the bit string.
  
  Args:
    arch: an int representing the bit string.
  """
  total = 0
  for _ in range(DIM):
    total += arch & 1
    arch = (arch >> 1)
  return total

In [8]:
import random

def random_architecture():
  """Returns a random architecture (bit-string) represented as an int."""
  return random.randint(0, 2**DIM - 1)

def mutate_arch(parent_arch):
  """Computes the architecture for a child of the given parent architecture.

  The parent architecture is cloned and mutated to produce the child
  architecture. The child architecture is mutated by flipping a randomly chosen
  bit in its bit-string.

  Args:
    parent_arch: an int representing the architecture (bit-string) of the
        parent.

  Returns:
    An int representing the architecture (bit-string) of the child.
  """

  position = random.randint(0, DIM - 1)  # Index of the bit to flip.
  
  # Flip the bit at position `position` in `child_arch`.
  child_arch = parent_arch ^ (1 << position)
  
  return child_arch
print(len(random_architecture()))

TypeError: object of type 'int' has no len()

In [3]:
import collections
import random

def ega_evolution(cycles, population_size, sample_size):
    population = []
    history = []  # Not used by the algorithm, only used to report results.

    # Initialize the population with random models.
    while len(population) < population_size:
        model = Model()
        model.arch = random_architecture()
        model.accuracy = train_and_eval(model.arch)
        # Find place of new model in the array, ordered from high to low accuracy
        i = 0
        while i < len(population):
            if population[i].accuracy < model.accuracy:
                population = population[:i] + [model] + population[:i+1]
    history = population

  # Carry out evolution in cycles. Each cycle produces a model and removes
  # another.
    while len(history) < cycles:
    # Sample randomly chosen models from the current population.
        sample = []
        while len(sample) < sample_size:
            candidate = random.choice(list(population))
            sample.append(candidate)

    # The parent is the best model in the sample.
    parent = max(sample, key=lambda i: i.accuracy)

    # Create the child model and store it.
    child = Model()
    child.arch = mutate_arch(parent.arch)
    child.accuracy = train_and_eval(child.arch)
    i = 0
    while i < len(population):
        if population[i].accuracy < child.accuracy:
            population = population[:i] + [child] + population[:i+1]
    history = population
    

    # Remove the worst model.
    population.popright()

    return history

In [4]:
history = ega_evolution(
    cycles=10, population_size=10, sample_size=5)

KeyboardInterrupt: 